# Firefly Voila Redshift 7 App

In [ ]:
import warnings

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', '.*astropy.extern.six will be removed.*')
    from astroquery.ned import Ned

In [ ]:
candidates = Ned.query_refcode('2012MNRAS.426.2772B')

In [ ]:
from IPython.display import IFrame, display

In [ ]:
import ipywidgets as widgets

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})

In [ ]:
import firefly_client

In [ ]:
import uuid
fc = firefly_client.FireflyClient(url='https://lsst-demo.ncsa.illinois.edu/firefly/',
                                  channel=uuid.uuid4().hex,
                                  html_file='slate.html')

In [ ]:
from astropy.table import Table

In [ ]:
surveys = Table.read('https://raw.githubusercontent.com/Caltech-IPAC/firefly/dev/src/firefly/' +
                     'java/edu/caltech/ipac/firefly/resources/irsa-image-master-table.csv',
                     format='csv')

In [ ]:
cosmos_surveys = surveys[surveys['missionId'] == 'COSMOS']

In [ ]:
import numpy as np

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
import time

In [ ]:
irsacats = Table.read('https://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-scan?mode=ascii', format='ipac')
projects = np.unique(irsacats['projectshort'].data)
inx = irsacats['catname'] == 'gaia_dr2_source'

In [ ]:
def visualize_candidate(candidate_number=1, size_in_arcsec=400):
    fc.reinit_viewer()
    tbl_index = candidates['No.'] == candidate_number
    c = SkyCoord(ra=candidates['RA'][tbl_index].data[0]*u.deg,
                 dec=candidates['DEC'][tbl_index].data[0]*u.deg)
    target = '{};{};EQ_J2000'.format(c.ra.deg, c.dec.deg)
    time.sleep(1)
    fc.show_table(tbl_id='gaiaTbl', title='Gaia DR2 catalog',
             target_search_info=dict(catalogProject=irsacats[inx]['projectshort'][0],
                                     catalog=irsacats[inx]['catname'][0],
                                     position=target,
                                     SearchMethod='Box',
                                     size=size_in_arcsec),
             options=dict(removable=True, showUnits=False, showFilters=True),
             meta=dict(DEFAULT_SYMBOL='CIRCLE', DEFAULT_COLOR='#FFDD11'))
    rvstring = fc._create_rangevalues_standard(algorithm='asinh',
                                               asinh_q_value=6,
                                               upper_value=98.0)
    time.sleep(2)
    fc.show_fits(plot_id='z+', Title='z+', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_subaru', SurveyKeyBand='z+',
            WorldPt=target, SizeInDeg=size_in_arcsec/3600,
            ColorTable=1, plotGroupId=1, 
            ZoomType='ARCSEC_PER_SCREEN_PIX', ZoomArcsecPerScreenPix=0.3,
            RangeValues=rvstring, OverlayPosition=False);
    rvstring_Y = fc._create_rangevalues_standard(algorithm='asinh',
                                             asinh_q_value=6,
                                             lower_value=0.5,
                                             upper_value=95.0)
    #time.sleep(1)
    fc.show_fits(plot_id='Y', Title='Y', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_ultravista', SurveyKeyBand='Y',
            WorldPt=target, SizeInDeg=size_in_arcsec/3600,
            ColorTable=1, plotGroupId=1, 
            ZoomType='ARCSEC_PER_SCREEN_PIX', ZoomArcsecPerScreenPix=0.3,
            RangeValues=rvstring_Y, OverlayPosition=False);
    #time.sleep(1)
    fc.show_fits(plot_id='H', Title='H', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_ultravista', SurveyKeyBand='H',
            WorldPt=target, SizeInDeg=size_in_arcsec/3600,
            ColorTable=1, plotGroupId=1, 
            ZoomType='ARCSEC_PER_SCREEN_PIX', ZoomArcsecPerScreenPix=0.3,
            RangeValues=rvstring, OverlayPosition=False);
    #time.sleep(1)
    fc.show_fits(plot_id='K', Title='K', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_ultravista', SurveyKeyBand='K',
            WorldPt=target, SizeInDeg=size_in_arcsec/3600,
            ColorTable=1, plotGroupId=1, 
            ZoomType='ARCSEC_PER_SCREEN_PIX', ZoomArcsecPerScreenPix=0.3,
            RangeValues=rvstring, OverlayPosition=False);
    #time.sleep(1)
    rvstring_irac = fc._create_rangevalues_standard(algorithm='asinh',
                                                asinh_q_value=6,
                                                lower_value=0.2,
                                                upper_value=95.0)
    fc.show_fits(plot_id='IRAC1', Title='3.6um', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_irac', SurveyKeyBand='IRAC1',
            WorldPt=target, SizeInDeg=size_in_arcsec/3600,            
            ColorTable=1, plotGroupId=1, 
            ZoomType='ARCSEC_PER_SCREEN_PIX', ZoomArcsecPerScreenPix=0.3,
            RangeValues=rvstring_irac, OverlayPosition=False);
    #time.sleep(1)
    fc.show_fits(plot_id='IRAC2', Title='4.5um', Type='SERVICE', Service='ATLAS',
            SurveyKey='cosmos.cosmos_irac', SurveyKeyBand='IRAC1',
            WorldPt=target, SizeInDeg=size_in_arcsec/3600,
            ColorTable=1, plotGroupId=1, 
            ZoomType='ARCSEC_PER_SCREEN_PIX', ZoomArcsecPerScreenPix=0.3,
            RangeValues=rvstring_irac, OverlayPosition=False);
    fc.dispatch('ImagePlotCntlr.wcsMatch', payload=dict(#plotId='z+', 
                                                         matchType='Standard', 
                                                         lockMatch=True))

In [ ]:
options = []
for i in range(1, len(candidates)):
    row = candidates[i]
    optstring = ' '.join([row['Object Name'], str(row['RA']), str(row['DEC']), str(row['Redshift'])])
    options.append((optstring, i))


In [ ]:
w1 = widgets.Dropdown(
         options=options, value=2, description='Candidate:', disabled=False)
w2 = widgets.IntSlider(value=200, min=100, max=600, step=50)

In [ ]:
from ipywidgets import interact

In [ ]:
interact(visualize_candidate, candidate_number=w1, size_in_arcsec=w2);


In [ ]:
with out:
    display(IFrame(fc.get_firefly_url(), 1000, 700))

In [ ]:
out